In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
!pip install torch torchvision torchaudio  # Install PyTorch
!git clone https://github.com/ultralytics/yolov5  # Clone YOLOv5 repository
%cd yolov5
!pip install -r requirements.txt  # Install requirements


Cloning into 'yolov5'...
remote: Enumerating objects: 16512, done.
remote: Counting objects: 100% (104/104), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 16512 (delta 41), reused 47 (delta 15), pack-reused 16408
Receiving objects: 100% (16512/16512), 15.12 MiB | 12.52 MiB/s, done.
Resolving deltas: 100% (11306/11306), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [2]:
import zipfile
import os

# Replace 'path_to_your_zip_file' with the path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/Machine Learning Project/training.zip'

# Replace 'destination_folder' with the path where you want to unzip your files
destination_folder = '/content/Dataset'

# Create destination directory if it does not exist
os.makedirs(destination_folder, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_folder)


In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dataset_dir = '/content/Dataset'
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')


In [4]:
# Get a list of all image files
all_images = os.listdir(images_dir)

# Split the dataset into training and validation (80-20 split for example)
train_images, val_images = train_test_split(all_images, test_size=0.2, random_state=42)

# Function to move images and labels
def move_files(files, source_folder, dest_folder):
    for file in files:
        shutil.move(os.path.join(source_folder, file), dest_folder)
        label_file = file.replace('jpg', 'txt').replace('png', 'txt')  # Assuming image extensions are jpg or png
        shutil.move(os.path.join(source_folder.replace('images', 'labels'), label_file), dest_folder.replace('images', 'labels'))

# Create directories for the splits
train_dir = os.path.join(dataset_dir, 'images/train')
val_dir = os.path.join(dataset_dir, 'images/val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(train_dir.replace('images', 'labels'), exist_ok=True)
os.makedirs(val_dir.replace('images', 'labels'), exist_ok=True)

# Move the files
move_files(train_images, images_dir, train_dir)
move_files(val_images, images_dir, val_dir)


In [6]:
import torch
print(torch.cuda.is_available())


True


In [7]:
# Train YOLOv5 on custom dataset for a certain number of epochs
!python train.py --img 640 --batch 16 --epochs 50 --data /content/Dataset/dataset.yaml --weights yolov5s.pt


Streaming output truncated to the last 5000 lines.
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
      39/49      5.36G    0.02774     0.1508   0.006785        843        640:  74% 99/134 [00:57<00:24,  1.45it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
      39/49      5.36G    0.02774     0.1506   0.006778        677        640:  75% 100/134 [00:57<00:19,  1.78it/s]libpng warning: iCCP: profile 'ICC Profile': 'GRAY': Gray color space not permitted on RGB PNG
      39/49      5.36G    0.02777     0.1506   0.006776        677        640:  75% 101/134 [00:57<00:15,  2.14it/s]libpng warning: iCCP: known incorrect sRGB profile
      39/49      5.36G    0.02777     0.1506   0

In [9]:
# Run detection
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ../Dataset/images/val


detect: weights=['runs/train/exp/weights/best.pt'], source=../Dataset/images/val, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/537 /content/Dataset/images/val/1004.png: 640x576 10 rooms, 5 windows, 10 doors, 52.9ms
image 2/537 /content/Dataset/images/val/1005.png: 640x576 10 rooms, 5 windows, 10 doors, 11.5ms
image 3/537 /content/Dataset/images/val/1007.png: 608x640 15 rooms, 6 windows, 13 doors, 52.3ms
image 4/537 /content/Dataset/images/val

In [10]:
!python val.py --weights runs/train/exp/weights/best.pt --data /content/Dataset/dataset.yaml --img 640 --task val


val: data=/content/Dataset/dataset.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-290-gb2ffe055 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/Dataset/labels/val.cache... 537 images, 0 backgrounds, 0 corrupt: 100% 537/537 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   0% 0/17 [00:00<?, ?it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warn

## Introduction

In the field of architectural design and analysis, identifying and cataloging elements like rooms, windows, and doors is crucial. The project employs the YOLOv5 algorithm, noted for its speed and accuracy in image recognition tasks. The primary objective is to evaluate the algorithm's performance in recognizing architectural components directly from drawings.

## Data

The dataset consists of 537 architectural drawings, encompassing a total of 14,635 labeled instances across three distinct categories: room, window, and door. The performance of the model on this dataset is summarized below:

| Class   | Images | Instances | Precision | Recall | mAP50 | mAP50-95 |
|---------|--------|-----------|-----------|--------|-------|----------|
| All     | 537    | 14,635    | 0.879     | 0.720  | 0.781 | 0.560    |
| Room    | 537    | 5,404     | 0.909     | 0.773  | 0.820 | 0.592    |
| Window  | 537    | 3,733     | 0.871     | 0.691  | 0.761 | 0.463    |
| Door    | 537    | 5,498     | 0.858     | 0.695  | 0.760 | 0.500    |


## Methodology

The study utilized the YOLOv5 model in a straightforward approach, applying no data preprocessing to assess baseline capabilities in architectural element detection. The dataset was divided into training and validation sets, with the model training until performance metrics plateaued.

## Results

The YOLOv5 model exhibited robust performance, particularly in identifying room categories. The precision and mAP scores were highest for rooms, suggesting the model's effectiveness in this context. While the performance on windows and doors was slightly lower, it still indicated considerable success in recognizing these elements within architectural drawings.
